In [1]:
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
segment3_workflow4=pd.read_csv('segment3_workflowS4.csv')
segment3_workflow4.CASE_ID.mode()

0    1052275|402830|625246
dtype: object

In [3]:
segment3_workflow4.head()

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS
0,1067449|424909|657898,Release Offer,Release Offer,18/12/2018 11:36:07,KRIT,Faculty,5,N,3b. Offer stage,Part 2 - Application Line ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
1,1067449|424909|657898,OC - Satisfied - Other,OC - Satisfied - English language requirements,25/1/2019 15:18:53,KNAVIN,Academic Services,4,N,1. App to Assessment,Part 6 - Requirement Verify,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
2,1067449|424909|657898,Accept,Accept,6/2/2019 9:12:01,KRIT,Faculty,3,N,4. Post offer,Part 2 - Application Line ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
3,1021374|367597|573577,Release Offer,Release Offer,12/7/2018 10:10:52,ATKINSONM,Faculty,10,N,3b. Offer stage,Part 2 - Application Line ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,N
4,1021374|367597|573577,Accept,Accept,8/8/2018 12:14:54,ATKINSONM,Faculty,9,N,4. Post offer,Part 2 - Application Line ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,N


In [4]:
segment3_workflow4['WRKFLW_TSK_MAP_DETAILED'].nunique()

32

In [5]:
segment3_workflow4=segment3_workflow4.drop(['WRKFLW_TSK_MAP_GROUPED', 'PATH_CRDATETIME','ITEM_CRUSER','WRKFLW_ORG_TYPE_CRUSER',
                         'STATUS_ORDER','LATEST_STATUS','WRKFLW_PROCESS','QUERY_PART','REFERRAL_ID',
                         'APPLICATION_ID','STU_ID','STU_CITIZEN_CD','STU_CITIZEN_NM','STU_BIRTH_DT',
                         'STU_ABOR_TSI_CD','STU_ABOR_TSI_NM','STU_BIRTH_CNTRY_CD','STU_BIRTH_CNTRY_NM',
                         'STU_CTZN_CNTRY_CD','STU_CTZN_CNTRY_NM','SUBMISSION_METHOD','ENTRY_TYPE',
                         'ASSESSMENT_DECISION_SCORE','ASSESSMENT_DECISION_NM','ASSESSMENT_OUTCOME_NM',
                         'LAST_COUNTRY_EDUCATION','PREV_INST_NM_MAIN','PREV_INST_NM_ALT1','PREV_INST_NM_ALT2',
                        'PREFERENCE_NO','SPK_CD','SPK_FULL_TITLE','ORG_UNIT_CD','AVAIL_ORG_UNIT_NM','SUPPORTING_ORG',
                        'STUDY_MODE_CD','RETURNING_DEFERRAL','CURR_AVAIL_YR','CURR_STUDY_PERIOD','CURR_SPRD_CD',
                        'CURR_SPRD_NM','ORIG_AVAIL_YR','ORIG_STUDY_PERIOD','ORIG_SPRD_CD','ORIG_SPRD_NM',
                         'SPK_CAT_TYPE_CD','BROAD_COURSE_LEVEL','DEGREETYPE','LIABILITY_CATEGORY_CD_RECODE',
                        'ORIG_LIABILITY_CATEGORY_CD_RECODE','OFFER_RELEASED_FG','OFFER_OUTCOME_CD','OFFER_CONDITIONS',
                        'APPLICATION_LINE_ID'], axis=1)
segment3_workflow4.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
0,1067449|424909|657898,Release Offer
1,1067449|424909|657898,OC - Satisfied - English language requirements
2,1067449|424909|657898,Accept
3,1021374|367597|573577,Release Offer
4,1021374|367597|573577,Accept


In [6]:
numOfWorkflowProcesses=segment3_workflow4.groupby('CASE_ID').CASE_ID.count()
numOfWorkflowProcesses.head()

CASE_ID
1000727|431447|668147    2
1001829|336678|526826    2
1001877|336757|526956    1
1002271|337186|527523    2
1007394|346044|540306    2
Name: CASE_ID, dtype: int64

In [7]:
counts = segment3_workflow4['CASE_ID'].value_counts()

In [8]:
counts.head()

1052275|402830|625246    7
1152919|538301|823869    6
954468|360490|561961     6
1025305|366531|571922    6
1076747|476620|731205    5
Name: CASE_ID, dtype: int64

In [47]:
filtered = counts[counts > 3]
seg3_count1=segment3_workflow4[segment3_workflow4['CASE_ID'].isin(filtered.index)]

In [48]:
seg3_count1['CASE_ID'].nunique()

19

In [49]:
seg3_count1.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
26,1072839|432324|669486,Release Offer
27,1072839|432324|669486,Release Triage Hold
28,1072839|432324|669486,Send for Assessment
29,1072839|432324|669486,Accept
66,1152919|538301|823869,Release Offer


In [50]:
y = pd.get_dummies(seg3_count1.WRKFLW_TSK_MAP_DETAILED)
y=y[:0]

In [51]:
y

,AR - Created - Assessment information (Coursework) (1),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - English language proficiency,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),Accept,Assessment Decision - No Decision,Assessment Decision - Qualified,Back to Perform Assessment,Decline,Defer,End My Assessment,Manage Acceptance,OC - Created - English language requirements,OC - Created - Evidence of course completion,OC - Satisfied - English language requirements,OC - Satisfied - Evidence of course completion,OC - Satisfied - MYEQUALS transcript authentication for students,PREPARE TO LOCK,Put Assessment On-Hold,Release Assessment Hold,Release Offer,Release Triage Hold,Restart Offer Workflow,Send for Assessment


In [52]:
y["CASE_ID"]=""

In [53]:
col_name="CASE_ID"
first_col = y.pop(col_name)

In [54]:
y.insert(0, col_name, first_col)

In [55]:
y['CASE_ID']=seg3_count1['CASE_ID'].unique()

In [56]:
import numpy as np
y = y.replace(np.nan,0, regex=True)
y.head()

,CASE_ID,AR - Created - Assessment information (Coursework) (1),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - English language proficiency,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),Accept,Assessment Decision - No Decision,Assessment Decision - Qualified,Back to Perform Assessment,Decline,Defer,End My Assessment,Manage Acceptance,OC - Created - English language requirements,OC - Created - Evidence of course completion,OC - Satisfied - English language requirements,OC - Satisfied - Evidence of course completion,OC - Satisfied - MYEQUALS transcript authentication for students,PREPARE TO LOCK,Put Assessment On-Hold,Release Assessment Hold,Release Offer,Release Triage Hold,Restart Offer Workflow,Send for Assessment
0,1072839|432324|669486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1152919|538301|823869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1027704|369135|575961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1052275|402830|625246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1047787|394823|614448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
dict_1=seg3_count1.to_dict(orient='list')

In [58]:
case_id = dict_1['CASE_ID']

In [59]:
workflow = dict_1['WRKFLW_TSK_MAP_DETAILED']

In [60]:
print(type(workflow))
print(len(workflow))

<class 'list'>
88


In [61]:
from collections import defaultdict

case_wf_dict = defaultdict(list)

for i in range(len(case_id)):
    item = case_id[i]
    case_wf_dict[item].append(workflow[i])
print(len(case_wf_dict))

19


In [62]:
y = y.set_index('CASE_ID')

In [63]:
y.head()

,AR - Created - Assessment information (Coursework) (1),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - English language proficiency,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),Accept,Assessment Decision - No Decision,Assessment Decision - Qualified,Back to Perform Assessment,Decline,Defer,End My Assessment,Manage Acceptance,OC - Created - English language requirements,OC - Created - Evidence of course completion,OC - Satisfied - English language requirements,OC - Satisfied - Evidence of course completion,OC - Satisfied - MYEQUALS transcript authentication for students,PREPARE TO LOCK,Put Assessment On-Hold,Release Assessment Hold,Release Offer,Release Triage Hold,Restart Offer Workflow,Send for Assessment
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
1072839|432324|669486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1152919|538301|823869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1027704|369135|575961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1052275|402830|625246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1047787|394823|614448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
y_col_name = y.columns.values.tolist()
print(y_col_name)

['AR - Created - Assessment information (Coursework) (1)', 'AR - Satisfied - Academic results for prior tertiary studies', 'AR - Satisfied - Advanced Standing', 'AR - Satisfied - Current enrolment at the University of Melbourne', 'AR - Satisfied - English language proficiency', 'AR - Satisfied - Graduate Access Melbourne', 'AR - Satisfied - Official syllabus or subject descriptions (in English)', 'Accept', 'Assessment Decision - No Decision', 'Assessment Decision - Qualified', 'Back to Perform Assessment', 'Decline', 'Defer', 'End My Assessment', 'Manage Acceptance', 'OC - Created - English language requirements', 'OC - Created - Evidence of course completion', 'OC - Satisfied - English language requirements', 'OC - Satisfied - Evidence of course completion', 'OC - Satisfied - MYEQUALS transcript authentication for students', 'PREPARE TO LOCK', 'Put Assessment On-Hold', 'Release Assessment Hold', 'Release Offer', 'Release Triage Hold', 'Restart Offer Workflow', 'Send for Assessment']


In [65]:
for key in case_wf_dict.keys():
    for process in case_wf_dict[key]:
        if process in y_col_name:
            y.loc[key, process] = 1
        else:
            continue

In [66]:
pd.set_option('display.max_rows', None)

In [67]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [68]:
y.head()

,AR - Created - Assessment information (Coursework) (1),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - English language proficiency,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),Accept,Assessment Decision - No Decision,Assessment Decision - Qualified,Back to Perform Assessment,Decline,Defer,End My Assessment,Manage Acceptance,OC - Created - English language requirements,OC - Created - Evidence of course completion,OC - Satisfied - English language requirements,OC - Satisfied - Evidence of course completion,OC - Satisfied - MYEQUALS transcript authentication for students,PREPARE TO LOCK,Put Assessment On-Hold,Release Assessment Hold,Release Offer,Release Triage Hold,Restart Offer Workflow,Send for Assessment
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
1072839|432324|669486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1152919|538301|823869,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1027704|369135|575961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1052275|402830|625246,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1047787|394823|614448,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [75]:
y = movecol(y, 
             cols_to_move=['Accept'], 
             ref_col='Send for Assessment',
             place='After')

In [76]:
y = movecol(y, 
             cols_to_move=['Defer'], 
             ref_col='Send for Assessment',
             place='After')

In [77]:
pd.set_option('display.max_columns',None)

In [78]:
y.head()

,AR - Created - Assessment information (Coursework) (1),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - English language proficiency,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Official syllabus or subject descriptions (in English),Assessment Decision - No Decision,Assessment Decision - Qualified,Back to Perform Assessment,Decline,End My Assessment,Manage Acceptance,OC - Created - English language requirements,OC - Created - Evidence of course completion,OC - Satisfied - English language requirements,OC - Satisfied - Evidence of course completion,OC - Satisfied - MYEQUALS transcript authentication for students,PREPARE TO LOCK,Put Assessment On-Hold,Release Assessment Hold,Release Offer,Release Triage Hold,Restart Offer Workflow,Send for Assessment,Defer,Accept
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
1072839|432324|669486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
1152919|538301|823869,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1027704|369135|575961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1052275|402830|625246,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1047787|394823|614448,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [79]:
y.shape

(19, 27)

In [80]:
y=y.fillna(0.0)

In [85]:
X=y.iloc[:,0:25].values
Y=y.iloc[:,25].values

In [86]:
#spliting dataset
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size = 0.3,random_state=0)

In [87]:
# K Nearest Neighbors
Knn = KNeighborsClassifier(n_neighbors=5)
Knn.fit(x_train,y_train)

KNeighborsClassifier()

In [88]:
knn_predict=Knn.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, knn_predict)*100,2)))

accuracy: 83.33%
